In [ ]:
import numpy as np
import matplotlib as mpl
import pylab
from libtiff import TIFFfile
from libtiff import TIFF
import os
from scipy import interpolate
import cPickle
import scipy.ndimage as ndimage
[]

In [ ]:
LineArraysFile=open("Pkls/TrimmedLineArraysBGSub.pkl")
LineArrays=cPickle.load(LineArraysFile)
XSize=118

In [ ]:
def AssembleLines(LineArraysIn):
    LensAll=[]
    AvesAll=[]
    for i in LineArraysIn:
        LensAll.append(len(i[0]))
        AvesAll.append(np.mean(i))
    YSize=len(LineArraysIn[0])
    BGVal=np.mean(AvesAll)
    MaxLen=max(LensAll)
    FullArray=np.ndarray(shape=(len(LineArraysIn)*YSize,MaxLen))
    for i in range(0,len(LineArraysIn)):
        for a in range(0,len(LineArraysIn[i][0])):
            for j in range(0,YSize):
                FullArray[int(i*YSize+j),a]=LineArraysIn[i][int(YSize-j-1),a]
        for a in range(len(LineArraysIn[i][0]),max(LensAll)):
            for j in range(0,YSize):
                FullArray[i*YSize+j,a]=BGVal
    return FullArray






In [ ]:
wpsleftall=[]
wpsrightall=[]
lengthsall=[]
dividersall=[]
for i in range(0,len(LineArrays)):
    wpsleft=[]
    wpsright=[]
    wp=np.loadtxt("./WordPos/WordPos"+str(i)+".csv",delimiter=',')
    pylab.imshow(LineArrays[i][::-1],cmap='binary')
    first=True
    for wordedge in wp:
        if(first):
            wpsleft.append(wordedge[0])
            first=False
        else:
            wpsright.append(wordedge[0])
            first=True
            pylab.plot([wpsleft[-1],wpsright[-1]],[150,150],color='pink',linewidth=2)
    pylab.show()
    wpsleftall.append(wpsleft)
    wpsrightall.append(wpsright)
    lengthsall.append(np.array(wpsright)-np.array(wpsleft))
    dividersall.append(np.array(wpsleft[1:]+wpsright[:-1])/2.)

In [ ]:
pylab.figure(figsize=(5,2))
pylab.imshow(AssembleLines(LineArrays),cmap='binary')
for i in range(0,len(wpsleftall)):
    for j in range(0,len(wpsleftall[i])):
        pylab.plot([wpsleftall[i][j],wpsrightall[i][j]],[85+220*i,85+220*i],'o-',color='pink',linewidth=2.,alpha=1)
#    pylab.plot([wpsleftall[1][-1],wpsrightall[1][-1]],[85+170,85+170],'o-',color='skyblue',linewidth=2.)
pylab.savefig("Plots/WordLimitsMarked.png",dpi=200)


In [ ]:
CharWidthRef=-(wpsleftall[1][-1]-wpsrightall[1][-1])/2.

In [ ]:
meandiscrepancy=[]
CharOptions=range(100,135)
for charlength in CharOptions:
    WordLetters=[]
    for i in range(0,len(wpsleftall)):
        WordLetters.append([])
        #print ("Line "+str(i))
        for j in range(0,len(wpsleftall[i])):
            NumLetters=(wpsrightall[i][j]-wpsleftall[i][j])/charlength +1
            #print(" Word " +str(j)+ " length = " +str(NumLetters))

            WordLetters[-1].append(NumLetters)
    FlatWordLengths=[item for sublist in WordLetters for item in sublist]


    meandiscrepancy.append(np.mean(np.array(FlatWordLengths)-np.round(np.array(FlatWordLengths))  ))

In [ ]:
BestFitCharWidth=CharOptions[np.argmin(abs(np.array(meandiscrepancy)))]

In [ ]:
BestFitCharWidth

In [ ]:
pylab.figure(figsize=(4,4))
pylab.plot(CharOptions,meandiscrepancy)
pylab.plot([BestFitCharWidth,BestFitCharWidth],[-0.2,0],'--',color='DarkGreen')
pylab.xlim(100,135)
pylab.plot([100,BestFitCharWidth],[0,0],'--',color='DarkGreen')
pylab.xlabel("Hypothetical letter width / pix")
pylab.ylabel("Mean Alignment Discrepancy")
pylab.ylim(-0.2,0.15)

In [ ]:
    charlength=118
    WordLetters=[]
    for i in range(0,len(wpsleftall)):
        WordLetters.append([])
        #print ("Line "+str(i))
        for j in range(0,len(wpsleftall[i])):
            NumLetters=(wpsrightall[i][j]-wpsleftall[i][j])/BestFitCharWidth +1
            #print(" Word " +str(j)+ " length = " +str(NumLetters))

            WordLetters[-1].append(NumLetters)

In [ ]:
FlatWPsLeft=[item for sublist in wpsleftall for item in sublist]
FlatWPsRight=[item for sublist in wpsrightall for item in sublist]
FlatWordLengths=[item for sublist in WordLetters for item in sublist]
MidPoints=(np.array(FlatWPsLeft)+np.array(FlatWPsRight))/2.

In [ ]:
#pylab.plot(MidPoints, np.array(FlatWordLengths)-np.round(np.array(FlatWordLengths)),'o')
#pylab.xlabel("X position")
#pylab.ylabel("distance to integer")
pylab.figure(figsize=(4,4))
pylab.hist(np.array(FlatWordLengths)-np.round(np.array(FlatWordLengths)))
pylab.xlabel("Distance from Integer")
pylab.ylabel("Counts")

In [ ]:
#Word end localization scale (/sqrt(2) due to 2 independent measurements)
np.std(np.array(FlatWordLengths)-np.round(np.array(FlatWordLengths)))/np.sqrt(2)


In [ ]:
LetterImgs=[]
\for i in range(0,len(wpsleftall)):
    LetterImgs.append([])
    for j in range(0,len(wpsleftall[i])):
        LetterImgs[-1].append([])
        NumLetters=int(round(WordLetters[i][j]))
        LetterCenters=np.linspace(wpsleftall[i][j],wpsrightall[i][j],NumLetters)
        for c in LetterCenters:
            Lo=int(c)-XSize/2
            Hi=int(c)+XSize/2
            Letter=LineArrays[i][:,Lo:Hi][::-1]
            Blurred=ndimage.gaussian_filter(Letter, sigma=5, order=0)
            MinVal=np.min(Blurred)
            MaxVal=np.max(Blurred)
            SubArray=(Blurred-MinVal)/(MaxVal-MinVal)
           # for x in range(len(SubArray)-20,len(SubArray)):
           #     for y in range(len(SubArray[0])-20,len(SubArray[0])):
           #         SubArray[x][y]=0 
            LetterImgs[-1][-1].append(SubArray)

In [ ]:
Line=2
Word=-1
Letter=2
pylab.imshow(LetterImgs[Line][Word][Letter],cmap='binary')
pylab.show()

Line=2
Word=-2
Letter=1
pylab.imshow(LetterImgs[Line][Word][Letter],cmap='binary')
pylab.show()

Line=2
Word=-3
Letter=2
pylab.imshow(LetterImgs[Line][Word][Letter],cmap='binary')
pylab.show()

Line=2
Word=-3
Letter=3
pylab.imshow(LetterImgs[Line][Word][Letter],cmap='binary')
pylab.show()

In [ ]:
LettersFile=open("Pkls/ExtracredLettersFromMemo.pkl",'wb')
cPickle.dump(LetterImgs,LettersFile)
LettersFile.close()


In [ ]:
pylab.figure(figsize=(5,5),dpi=200)
for l in range(0,len(LineArrays)):
    SummedIntensity=[]
    for i in range(0,len(LineArrays[l])):
        SummedIntensity.append(sum(LineArrays[l][i])/len(LineArrays[l][i]))
    pylab.plot(SummedIntensity,'-',label='Line '+str(l))
    pylab.legend(loc='upper left')
pylab.plot([85,85],[0,60],'--',color='black')
pylab.plot([205,205],[0,60],'--',color='black')
pylab.plot([145,144],[0,60],':',color='black')



pylab.ylim(20,60)
pylab.savefig("Plots/CharHeights.png",dpi=200)
pylab.xlabel("Height within line")
pylab.ylabel("Average Intensity")


In [ ]:
pylab.hist(np.ndarray.flatten(LetterImgs[1][1][1]),bins=np.arange(0,1,0.01))

In [ ]:
(85.+205.)/2

In [ ]:
YSize